In [1]:
import pandas as pd
import numpy as np
import os
# from searcher.csv_embedding_searcher import CSVEmbeddingSearcher

from sentence_transformers import SentenceTransformer, util

In [ ]:

class CSVEmbeddingSearcher:
    def __init__(
        self,
        csv_path: str,
        embedding_column: str,
        model_name: str = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    ):
        """
        :param csv_path: Путь к CSV-файлу.
        :param embedding_column: Название столбца, значение которого будем преобразовывать в вектор.
        :param model_name: Название модели Sentence Transformers.
        """
        self.csv_path = csv_path
        self.embedding_column = embedding_column
        self.model_name = model_name
        
        df = pd.read_csv(self.csv_path, sep=";", header=0)
        
    

        # 3) Удаляем строки, у которых в первой колонке (df.columns[0]) отсутствует значение (NaN)
        # df.dropna(subset=[df.columns[0]], inplace=True)
        
        # (Опционально, если нужно также убрать случаи, когда 
        # значение в первой колонке — пустая строка)
        # df = df[df[df.columns[0]].astype(str).str.strip() != ""]
        # self.df = df
        
        self.model = SentenceTransformer(self.model_name)
        
        # Создадим 6-й столбец с эмбеддингами
        self._create_embedding_column()
    
    def _create_embedding_column(self):
        """
        Приватный метод: проходится по датасету,
        кодирует значение из embedding_column в вектор
        и кладёт в новый столбец 'embedding_vector'.
        """
        if self.embedding_column not in self.df.columns:
            raise ValueError(f"Столбец '{self.embedding_column}' не найден в первых 5 колонках CSV.")
        
        embeddings_list = []
        
        # Кодируем каждую строку — для больших объёмов лучше использовать batch_size, но здесь для наглядности всё итеративно.
        for _, row in self.df.iterrows():
            text_value = str(row[self.embedding_column])  # на случай, если там не строка
            emb = self.model.encode(text_value, convert_to_numpy=True)
            embeddings_list.append(emb)
        
        self.df["embedding_vector"] = embeddings_list
    
    def search_nearest(self, query: str, top_k: int = 1):
        """
        Ищет в датафрейме ближайшие по косинусному сходству векторы (колонка 'embedding_vector')
        для заданного query. Возвращает top_k совпадений с их метаданными (индекс, похожесть).
        
        :param query: Текст для поиска ближайшего совпадения.
        :param top_k: Сколько ближайших совпадений вернуть.
        :return: Список словарей вида:
            [
              {
                "index": индекс_в_исходном_df,
                "similarity": сходство,
                "row_data": <запись датафрейма целиком>
              },
              ...
            ]
        """
        # Генерируем вектор для запроса
        query_emb = self.model.encode(query, convert_to_tensor=True)
        
        # Собираем все эмбеддинги из DataFrame в один массив
        all_emb = np.vstack(self.df["embedding_vector"].values)
        
        # Вычисляем косинусное сходство разом
        similarities = util.pytorch_cos_sim(query_emb, all_emb)[0].cpu().numpy()  # shape: [num_rows]
        
        # Отсортируем индексы по убыванию сходства
        sorted_indices = np.argsort(-similarities)
        
        # Возьмём top_k результатов
        top_indices = sorted_indices[:top_k]
        
        results = []
        for idx in top_indices:
            sim_score = float(similarities[idx])
            row_data = self.df.iloc[idx].to_dict()
            
            results.append({
                "index": idx,
                "similarity": sim_score,
                "Наименование": row_data["Наименование"],
                "Подкласс 2": row_data["Подкласс 2"],
            })
        
        return results
    
    def get_stats(self):
        """
        Возвращает простую статистику:
         - время загрузки модели,
         - время на создание эмбеддингов,
         - текущее количество строк и столбцов в датафрейме.
        """
        return {
            "model_name": self.model_name,
            "model_load_time": self.model_load_time,
            "embedding_build_time": getattr(self, "embedding_build_time", None),
            "data_shape": self.df.shape
        }



In [3]:
df = pd.read_csv('searcher\\data\\06_Com.csv', encoding='utf-8', sep=';')

In [4]:
df_class = df.dropna(subset=['Класс'])
df_subclass1 = df.dropna(subset=['Подкласс 1'])
df_subclass2 = df.dropna(subset=['Подкласс 2'])
df_type = df.dropna(subset=['Тип (%%)'])


In [5]:
df_subclass2.to_csv('searcher\\data\\06_Com_subclass2.csv', encoding='utf-8', sep=';', index=False)

In [6]:
df_subclass2.head(5)

,Класс,Подкласс 1,Подкласс 2,Тип (%%),Наименование,Определение компонента,Источник,Критерии определения подклассов,Примеры компонентов mпо ISO 81346-2:2019,Примеры компонентов(НТД РФ),Синхронизация,Unnamed: 11,Unnamed: 12,Unnamed: 13,Примечание,Синонимы,УИН
3,NaN,NaN,BAA,NaN,измерительный трансформатор напряжения,объект для измерения электрического потенциала...,ГОСТ IEC 61869-3-2012,NaN,конденсатор связи\r\nизмерительный трансформат...,NaN,IfcTransformer,NaN,NaN,NaN,NaN,NaN,KSI060000004
4,NaN,NaN,BAB,NaN,реле напряжения,объект индикации наличия электрического потенц...,ГОСТ IEC 60050-447-2014,NaN,реле измерения напряжения,реле напряжения,"IfcSensor, IfcFlowInstrument",89.1.62.01-0822\r\n89.1.62.01-0823\r\n89.1.62....,NaN,NaN,NaN,NaN,KSI060000005
6,NaN,NaN,BBA,NaN,датчик электрического сопротивления,объект для измерения удельного сопротивления э...,Приказ Москомэкспертизы от 28.12.2022 N МКЭ-ОД...,NaN,NaN,NaN,IfcSensor,NaN,ЭЛ 40 55 20 10,NaN,NaN,NaN,KSI060000007
7,NaN,NaN,BBB,NaN,детектор электрического сопротивления,объект индикации наличия удельного сопротивлен...,Приказ Москомэкспертизы от 28.12.2022 N МКЭ-ОД...,NaN,NaN,NaN,IfcSensor,NaN,ЭЛ 40 55 20 17,NaN,NaN,NaN,KSI060000008
8,NaN,NaN,BBC,NaN,датчик термического сопротивления,объект для измерения сопротивления теплового п...,Приказ Москомэкспертизы от 28.12.2022 N МКЭ-ОД...,NaN,NaN,NaN,IfcSensor,NaN,ЭЛ 40 55 20 10,NaN,NaN,NaN,KSI060000009


In [7]:

searcher = CSVEmbeddingSearcher("searcher\\data\\06_Com_subclass2.csv", "Наименование")

In [9]:
query_text = "Дверь"


result = searcher.search_nearest(query_text, 5)
print(result)

ValueError: need at least one array to concatenate